# Importing modules

In [34]:
import warnings
warnings.filterwarnings("ignore")
import speech_recognition as sr
import pyaudio
from googletrans import Translator
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import pytextrank
from rake_nltk import Rake
import re, string
import en_core_web_lg
import time
from math import *

# Transcribing an audio file

In [14]:
r=sr.Recognizer()

In [27]:
audiofile=sr.AudioFile('harvard.wav')
with audiofile as source:
    audio=r.record(source)
    #audio=r.record(source,duration=6)
queries = r.recognize_google(audio,show_all=True)
queries

{'alternative': [{'transcript': 'the salesman old beer drinkers it takes hi to bring out the order I call dip restore selfinvest a salt a call this find him because of my favourite is just for food is Bihar cross bun',
   'confidence': 0.84945077},
  {'transcript': 'the salesman old beer drinkers it takes hi to bring out the order I called research help in West a salt a call this find him because of my favourite is just for food is Bihar cross bun'},
  {'transcript': 'the salesman old beer drinkers it takes hi to bring out the older I call dip research help in West a salt a call this find him because of my favourite is just for food is Bihar cross bun'},
  {'transcript': 'the salesman old beer drinkers it takes hi to bring out the older I called research help in West a salt a call this find him because of my favourite is just for food is Bihar cross bun'},
  {'transcript': 'the salesman old beer drinkers it takes hi to bring out the order I called research help invest a salt a call thi

In [28]:
query = queries["alternative"][0]["transcript"]
query

'the salesman old beer drinkers it takes hi to bring out the order I call dip restore selfinvest a salt a call this find him because of my favourite is just for food is Bihar cross bun'

# Using the microphone

In [16]:
'''r = sr.Recognizer()
m = sr.Microphone()

try:
    with m as source: r.adjust_for_ambient_noise(source)
    print("Set minimum energy threshold to {}".format(r.energy_threshold))
    print("Say something!")
    with m as source: audio = r.listen(source)
    #print(audio)
    print("Got it! Now to recognize it...")
    try:
        value = r.recognize_google(audio,show_all=False,language = 'en-US') #fr-FR, hi-IN,kn-IN,ta-IN
        #value = r.recognize_sphinx(audio)
        #print("You said {}".format(value))
        print(value)
    except sr.UnknownValueError:
        print("Didn't catch that")
except KeyboardInterrupt:
    pass'''

'r = sr.Recognizer()\nm = sr.Microphone()\n\ntry:\n    with m as source: r.adjust_for_ambient_noise(source)\n    print("Set minimum energy threshold to {}".format(r.energy_threshold))\n    print("Say something!")\n    with m as source: audio = r.listen(source)\n    #print(audio)\n    print("Got it! Now to recognize it...")\n    try:\n        value = r.recognize_google(audio,show_all=False,language = \'en-US\') #fr-FR, hi-IN,kn-IN,ta-IN\n        #value = r.recognize_sphinx(audio)\n        #print("You said {}".format(value))\n        print(value)\n    except sr.UnknownValueError:\n        print("Didn\'t catch that")\nexcept KeyboardInterrupt:\n    pass'

# Translating the text to English

In [81]:
translator = Translator()
source_details = translator.detect(query)
print(source_details)
if source_details.lang!='en':
    query = translator.translate(query, src = source_details.lang, dest = 'en')
    print(query)
    print(query.text)
else:
    print(query)

Detected(lang=en, confidence=1.0)
the salesman old beer drinkers it takes hi to bring out the order I call dip restore selfinvest a salt a call this find him because of my favourite is just for food is Bihar cross bun


# Keyword Extraction

In [82]:
lemmatizer = WordNetLemmatizer()
stopw = set(stopwords.words('english'))
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
sp = en_core_web_lg.load()
r = Rake()
textrank = pytextrank.TextRank()
sp.add_pipe(textrank.PipelineComponent, name="textrank", last=True)

In [83]:
contraction = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot", 
    "can't've": "cannot have", 
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
    "hadn't've": "had not have", 
    "hasn't": "has not",
    "haven't": "have not", 
    "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
    "he'll've": "he he will have", "he's": "he is", "how'd": "how did", 
    "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
    "I'll've": "I will have","I'm": "I am", "I've": "I have", 
    "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
    "i'll've": "i will have","i'm": "i am", "i've": "i have", 
    "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
    "it'll": "it will", "it'll've": "it will have","it's": "it is", 
    "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
    "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
    "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
    "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
    "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
    "she's": "she is", "should've": "should have", "shouldn't": "should not", 
    "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
    "this's": "this is",
    "that'd": "that would", "that'd've": "that would have","that's": "that is", 
       "there'd": "there would", "there'd've": "there would have","there's": "there is", 
       "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
       "they'll've": "they will have", "they're": "they are", "they've": "they have", 
       "to've": "to have", "wasn't": "was not", "we'd": "we would", 
       "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
       "we're": "we are", "we've": "we have", "weren't": "were not", 
       "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
       "what's": "what is", "what've": "what have", "when's": "when is", 
       "when've": "when have", "where'd": "where did", "where's": "where is", 
       "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
       "who's": "who is", "who've": "who have", "why's": "why is", 
       "why've": "why have", "will've": "will have", "won't": "will not", 
       "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
       "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
       "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
       "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def clean(text):
    text = text.lower()
    temp = ""
    for i in text.split():
        try:
            temp+=contraction[i]+' '
        except:
            temp+= i+' '
    text = temp.strip()
    text = text.lower().translate(remove_punctuation_map)
    text = re.sub("[^a-zA-Z#]"," ",text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r",", "", text)
    text = re.sub(r"\.", "", text)
    text = re.sub(r"!", "!", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"'", "", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", ":", text)
    text = re.sub(r' +',' ',text)
    return text.strip()

def stopwordremoval(text):
    text = word_tokenize(text)
    text = [i for i in text if i not in stopw]
    return " ".join(text)

In [84]:
def pp_set(text, op):
    key_tokenized_sentences = sent_tokenize(text)
    key_tokenized_words = word_tokenize(text)
    if op == "token_sent":
        return key_tokenized_sentences
    elif op == "token_word":
        return key_tokenized_words
    elif op == "clean_sent":
        return [clean(i) for i in key_tokenized_sentences]
    elif op == "clean_word":
        return [clean(i) for i in key_tokenized_words]
    elif op == "lem_sent":
        key_clean_sentences = pp_set(text, "clean_sent")
        return [" ".join([lemmatizer.lemmatize(j) for j in i.split()]) for i in key_clean_sentences]
    elif op == "lem_word":
        key_clean_words = pp_set(text, "clean_word")
        return [lemmatizer.lemmatize(i) for i in key_clean_words]
    elif op == "prep_sent":
        key_clean_sentences = pp_set(text, "clean_sent")
        return [" ".join([i for i in j.split() if i not in stopw]) for j in key_clean_sentences]
    elif op == "prep_word":
        key_preprocessed_sentences = pp_set(text, "prep_sent")
        key_preprocessed_words = []
        for i in key_preprocessed_sentences:
            key_preprocessed_words.extend(word_tokenize(i))
        return key_preprocessed_words
    elif op == "pp_lem_word":
        return [lemmatizer.lemmatize(i) for i in pp_set(text, "prep_word")]

In [100]:
def extract_keywords(text):
    r.extract_keywords_from_sentences(pp_set(text, "lem_sent")) # Effectiveness : tokenized > lemmatized > clean 
    rake_keywords = r.get_ranked_phrases()
    spdoc = sp(text)
    ner_keywords = []
    for ent in spdoc.ents:
        ner_keywords.append(ent.text)
    spdoc = sp(" ".join(pp_set(text, "clean_word")))
    pytr_keywords = []
    for p in spdoc._.phrases:
        for term in p.chunks:
            if term.text not in pytr_keywords and term.text not in stopw:
                x = term.text
                pytr_keywords.append(x)
    nounchunk_keywords = list(set([i.text for i in set(spdoc.noun_chunks) if i.text not in stopw and len(min(i.text.split(),key = len))>1]))
    
    bigrams = list(ngrams(pp_set(text, "clean_word"),2))
    trigrams = list(ngrams(pp_set(text, "clean_word"),3))
    quadgrams = list(ngrams(pp_set(text, "clean_word"),4))
    pentagrams = list(ngrams(pp_set(text, "clean_word"),5))
    quadgrams = list(set([" ".join([j for j in i if j not in stopw]).strip() for i in quadgrams]))
    #quadgrams = [i for i in quadgrams if i not in stopw and i!='' and len(i.split())>1]
    ngram_vector_key = dict()
    key_clean_sentences = pp_set(text, "clean_sent")
    for i in quadgrams:
        ngram_vector_key[i] = [0 for i in range(len(key_clean_sentences))]
    for i in range(len(key_clean_sentences)):
        for phrase in ngram_vector_key:
            ngram_vector_key[phrase][i] = (key_clean_sentences[i].count(phrase)/len(word_tokenize(key_clean_sentences[i])))
            df = 0
            for j in key_clean_sentences:
                if phrase in j:
                    df+=1
            ngram_vector_key[phrase][i]*=(1+log((len(key_clean_sentences)+1)/(df+1)))
    ngram_keywords = sorted(ngram_vector_key,key = lambda x:sum(ngram_vector_key[x]),reverse = True)
    
    
    
    all_keywords = rake_keywords+pytr_keywords+ner_keywords+nounchunk_keywords#+ngram_keywords
    all_keywords = list(set(all_keywords))
    sorted_keywords = list(all_keywords)
    sorted_keywords.sort()
    for i in range(len(sorted_keywords)):
        sorted_keywords[i] = re.sub(r' +',' ',sorted_keywords[i])
    
    return sorted_keywords

In [101]:
def group(sorted_keywords):
    grouped_keys = []
    for i in sorted_keywords:
        if len(grouped_keys)==0:
            grouped_keys.append([i])
            continue
        else:
            flag = False
            for j in grouped_keys:
                if i in j:
                    flag = True
                    break
                temp1 = " ".join([lemmatizer.lemmatize(t) for t in stopwordremoval(i).split()])
                for k in j:
                    temp2 = " ".join([lemmatizer.lemmatize(t) for t in stopwordremoval(k).split()])
                    short = min(temp1,temp2)
                    long = max(temp1,temp2)
                    if short in long:
                        flag = True
                        j.append(i)
                        break
                if flag == True:
                    break            
            if flag==False:
                grouped_keys.append([i])
    temp = []
    for i in grouped_keys:
        k = sorted(i,key = len)
        temp.append(k)
    return temp

In [102]:
def remove_duplicates(grouped_keys):    
    for i in range(len(grouped_keys)):
        grouped_keys[i] = list(set(grouped_keys[i]))
        temp = list(grouped_keys[i])
        process_set = [" ".join([lemmatizer.lemmatize(l) for l in stopwordremoval(j).split()]) for j in grouped_keys[i]]
        process_set = list(set(process_set))
        for temp_key1 in grouped_keys[i]:
            x = " ".join([lemmatizer.lemmatize(k) for k in stopwordremoval(temp_key1).split()])
            if process_set.count(x)>1:
                temp.remove(temp_key1)   
        grouped_keys[i] = temp
        grouped_keys[i] = sorted(grouped_keys[i])
        
        
    for i in range(len(grouped_keys)):
        temp = list(grouped_keys[i])
        for j in range(len(grouped_keys[i])):
            word = grouped_keys[i][j]
            for k in temp:
                if word in k and word!=k:
                    temp.remove(word)
                    break
        grouped_keys[i] = sorted(temp,key = len, reverse = True)
    grouped_keys = [i for i in grouped_keys if len(i)>0]
    return grouped_keys

In [103]:
def finalize(grouped_keys):
    temp_keywords = []
    final_keywords = []
    for i in grouped_keys:
        for j in i:
            temp_keywords.append(j)
    
    temp_keywords = remove_duplicates(group(temp_keywords))
    
    for i in temp_keywords:
        for j in i:
            final_keywords.append(j)
    return final_keywords

In [104]:
keywords = finalize(remove_duplicates(group(extract_keywords(query.lower()))))
keywords

['a salt',
 'bihar cross bun',
 'bring',
 'call dip restore selfinvest',
 'my favourite',
 'find',
 'food',
 'the order',
 'the salesman old beer drinkers',
 'selfinvest',
 'take hi']